# Using Tools
-Define the tool (function) that we want to call

# Step 1: Define model and tool 

In [ ]:
import json
import requests
import instructor
from pydantic import BaseModel, Field
from openai import OpenAI

#Only for Jupyter Notebooks
import nest_asyncio
nest_asyncio.apply()

#--------------------------------------------------------
# Initialize the OpenAI client for Ollama   
# And add instructor wrapper for Pydantic validation
#--------------------------------------------------------
client = OpenAI(
    base_url="http://localhost:11434/v1/", 
    api_key=""
    )

client = instructor.patch(client)


#----------------------------------------------------------
# Define the tool (function) that we want to call
#----------------------------------------------------------
def get_weather(latitude, longitude):
    """This is a publically available API that returns the weather for a given location."""
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m"
    )
    data = response.json()
    return data["current"]


#--------------------------------------------------------
# Define the tool schema for the LLM
#--------------------------------------------------------
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current temperature for provided coordinates in celsius.",
            "parameters": {
                "type": "object",
                "properties": {
                    "latitude": {"type": "number"},
                    "longitude": {"type": "number"},
                },
                "required": ["latitude", "longitude"],
                "additionalProperties": False,
            },
            "strict": True,
        },
    }
]

#--------------------------------------------------------
# Define the response model for the LLM
#--------------------------------------------------------
class WeatherResponse(BaseModel):
    temperature: float = Field(description="The current temperature in celsius for the given location.")
    response: str = Field(description="A natural language response to the user's question.")


#--------------------------------------------------------
# Step 1: Call model with get_weather tool defined
#--------------------------------------------------------
model = "llama3.1:8b"
messages=[        
            {"role": "system",  "content": "You are a helpful weather assistant."},
            {"role": "user",    "content": "What's the weather like in Tainan, Taiwan now ?"},
        ]

completion = client.chat.completions.create(
    model=model,
    tools=tools,
    messages=messages,     
    temperature=0,
    response_model= WeatherResponse   
    )

print(json.dumps(completion.model_dump(), indent=2))

# No response_model= WeatherResponse 
completion2 = client.chat.completions.create(
    model=model,
    tools=tools,
    messages=messages,     
    temperature=0, 
    )

print(json.dumps(completion2.model_dump(), indent=2))


# Or just the model response
#print(completion.model_dump())


# Step 2: Execute get_weather function

In [ ]:
# Step 2: Execute get_weather function
for tool_call in completion2.choices[0].message.tool_calls:
    # Extract the function name and arguments
    name = tool_call.function.name    
    args = json.loads(tool_call.function.arguments)
    print(f"Calling function {name} with args {args}")
    
    messages.append(completion2.choices[0].message)
    print("Messages so far:", messages)
    
    # Call the function
    result = None
    if name == "get_weather":
        result = get_weather(**args)
        print(f"Function call result: {result}") 
                   
        messages.append(
            {"role": "tool", "tool_call_id": tool_call.id, "content": json.dumps(result)}
        )
        print("Messages so far:", messages)
        

# Step 3: Supply result and call model again

In [ ]:
# Step 3: Supply result and call model again


# Call the model again with the function result
completion = client.beta.chat.completions.parse(
    model=model,
    messages=messages,
    tools=tools,
    response_format=WeatherResponse,
)


final_response = completion.choices[0].message.parsed
print(final_response.temperature)
print(final_response.response)
print(final_response)

28.5
There has been a tool call. Today's weather in Tainan, Taiwan is mostly sunny with moderate temperatures around 28°C (82°F). The wind speed is relatively low at approximately 1.4 m/s (3.13 mph), indicating a calm environment.
temperature=28.5 response="There has been a tool call. Today's weather in Tainan, Taiwan is mostly sunny with moderate temperatures around 28°C (82°F). The wind speed is relatively low at approximately 1.4 m/s (3.13 mph), indicating a calm environment."
